In [2]:
from PIL import Image
import pandas as pd
from configs import configs
from tqdm import tqdm

In [2]:
train = pd.read_csv(configs['train_csv'])
val = pd.read_csv(configs['val_csv'])
test = pd.read_csv(configs['test_csv'])

df = pd.concat([train, val, test]).reset_index()
img_paths = df['img_path'].values

In [3]:
all_df = pd.read_csv(configs['mimic_csv_file_path'])
img_paths = all_df['img_path'].values
print(img_paths.shape)

(273585,)


In [4]:
def new_name(p):
    return ''.join(p.split('.')[:-1]) + '.256.' + p.split('.')[-1]

def new_path(p):
    name = p.split('/')[-1]
    return configs['mimic_dir'] + 'images256/' + name

In [6]:
img_paths[0]

'mimic_cxr/images512/p10/p10000032/s50414267/02aa804e-bde0afdd-112c0b34-7bc16630-4e384014.png'

In [5]:
all_df['img_path_256'] = all_df['img_path'].apply(new_path)

In [12]:
try:
    Image.open(all_df.iloc[150710]['img_path'])

    

file not found


In [14]:
file_not_found_id = []

for i, (p, new_p) in tqdm(enumerate(zip(all_df.iloc[150710:]['img_path'], all_df.iloc[150710:]['img_path_256']))):
    try:
        im = Image.open(p)
    except FileNotFoundError:
        print(f"index {i}: {p} not found")
        file_not_found_id.append(i+150710)
        continue
    im = im.resize((256, 256))
    im.save(new_p)

14it [00:00, 138.99it/s]

index 0: mimic_cxr/images512/p15/p15517908/s50255536/4ec69a3d-c26ee4b8-99157c7d-cad8faaf-4b0fc113.png not found
index 1: mimic_cxr/images512/p15/p15517908/s50255536/caea02bd-dcd9808d-d596f848-3ddf200a-037dd6d0.png not found
index 2: mimic_cxr/images512/p15/p15517908/s51770359/02798464-0d35345f-c4c6f074-9f2b8fcc-a27d7574.png not found
index 3: mimic_cxr/images512/p15/p15517908/s51770359/2ddc0f1e-f9f2b601-bebeb8c1-b96d3d8c-60051bdd.png not found
index 4: mimic_cxr/images512/p15/p15517908/s51770359/45be2b0d-ad5e7b0f-62861dc2-8f7a8f04-32b6d560.png not found
index 5: mimic_cxr/images512/p15/p15517908/s52506750/b913b602-e420f788-f8ae2e56-b4437002-c1953512.png not found


98749it [23:44, 78.06it/s]

index 98735: mimic_cxr/images512/p19/p19124994/s53492226/25806a33-744e8586-0cc8cdc8-ed83d82d-d7dee178.png not found
index 98737: mimic_cxr/images512/p19/p19124994/s53492226/d82fae25-9c068173-624b6e7f-5c5752c5-25f662e5.png not found


122875it [29:26, 69.57it/s]


In [19]:
file_not_found_id

[150710, 150711, 150712, 150713, 150714, 150715, 249445, 249447]

In [21]:
dropped = pd.concat([
    all_df.iloc[:150710],
    all_df.iloc[150716:249445],
    all_df.iloc[249446],
    all_df.iloc[249448:],
])

In [34]:
dropped.columns

Index([             0,   'No Finding',   'Unnamed: 0', 'ViewPosition',
           'dicom_id',     'img_path', 'img_path_256',       'report',
        'report_path',     'study_id',   'subject_id'],
      dtype='object')

In [36]:
dropped = dropped.drop(columns=[0, 'Unnamed: 0'], axis=1)

In [37]:
dropped.to_csv(configs['mimic_csv_file_path'], index=False)

In [19]:
for p in tqdm(img_paths):
    im = Image.open(p)
    im = im.resize((256, 256))
    new_p = new_name(p)
    im.save(new_p)

100%|██████████| 13679/13679 [02:46<00:00, 82.32it/s]


In [28]:
train['img_path_256'] = train['img_path'].apply(new_name)
val['img_path_256'] = val['img_path'].apply(new_name)
test['img_path_256'] = test['img_path'].apply(new_name)

In [32]:
train.to_csv(configs['train_csv'], index=False)
val.to_csv(configs['val_csv'], index=False)
test.to_csv(configs['test_csv'], index=False)